# 场景一 目标站点网页分析

In [5]:
## 任务一 向站点发送请求

### 步骤 1	导入网页请求库requests

# requests是Python中的HTTP库，在导入requests库之前，需确认在Python中已安装。
import requests
from requests.exceptions import RequestException


### 步骤 2	配置请求信息


# 通过requests.get(url,headers)方法请求网页，其中传入的参数是url和headers。
# 	url：目标站点地址信息；
# 	headers：浏览器信息，若不添加headers信息，服务器无法识别浏览器信息，则将该请求视为非法请求，无法获取正常的响应结果。最后打印出状态码status_code的返回结果。注意：每个浏览器的请求信息不同，需要根据实际情况传入该参数信息。
url = 'https://movie.douban.com/subject/27119724/comments?sort=new_score&status=P'
headers = {
        'User-Agent':'Mozilla/5.0(Macintosh;Intel Mac OS X 10_14_6)AppleWebKit/537.36(KHTML, like Gecko)Chrome/79.0.3945.130Safari/537.36' }
response = requests.get(url,headers = headers)
print(response.status_code)


## 任务二获取网页内容


# 获取网页内容。
# 	requests.get()方法请求返回参数text获取网页源代码内容。
html = response.text
print(type(response.text))


## 任务三 网页结构 解析


# 请根据网页内容在下表中填写采集数据字段用户名、评论时间、评论内容的标签信息：
# 字段	标签名称	标签类型
# user		
# time		
# comment		

# 【参考答案】
# 字段	标签名称	标签类型
# user	<a>	Null
# time	<span>	comment-time
# comment	<span>	short


## 任务四 构造网页获取函数


# 构造网页获取函数。
# 将以上代码封装为函数，以便于多次调用，自定义函数名称为get_page()，将url作为传入参数。在函数体中，使用if条件语句判断响应状态，若状态码为正常，则返回网页内容；若网页请求失败，则返回None。
def get_page(url):
    headers = {
        'User-Agent':'Mozilla/5.0(Macintosh;Intel Mac OS X 10_14_6)AppleWebKit/537.36(KHTML, like Gecko)Chrome/79.0.3945.130Safari/537.36'
        }
    try:
        response = requests.get(url,headers=headers)
        if response.status_code == 200:
            return response.text
        return None
    except RequestException:
        return None

200
<class 'str'>


# 场景二

## 任务一 网页解析

### 步骤 1	导入BeautifulSoup库

In [2]:

# 本次实验采用BeautifulSoup库对网页进行解析，相比于直接提取和正则表达式匹配的方法，
# 使用BeautifulSoup更加简洁、高效，导入前需确认已安装BeautifulSoup库。
from bs4 import BeautifulSoup


## 步骤 2	解析网页

In [6]:

# 声明BeautifulSoup对象为soup，并传入html代码和解析方式，BeautifulSoup提供以下4种解析库：
# 解析器	使用方法	说明
# Python标准库	BeautifulSoup(makeup,”html.parser”)	Python的内置标准库
# lxml HTML解析器	BeautifulSoup(makeup,”lxml”)	常用HTML代码解析器
# lxml XML解析器	BeautifulSoup(makeup,”xml”)	唯一支持XML的解析器
# html5lib	BeautifulSoup(makeup,”html5lib”)	以浏览器的方式解析文档并生成HTML5格式的文档
soup = BeautifulSoup(html, 'lxml')


## 任务二 网页数值提取

### 步骤 1	提取数据

In [7]:
# 	find_all()：查找标签，通过class_设置标签类型，class_="comment"和attrs={"class": "comment-info"}两种写法实现结果相同。
# 注意：class在Python中属于关键字， 因此class需要添加下划线。

items = soup.find_all("div", class_="comment")
for item in items:
    info = item.find_all(name='span', attrs={"class": "comment-info"})
    for users in info:
        user = users.find_all(name='a', attrs={"class": ""})[0].string


### 步骤 2	生成格式化数据

In [8]:

# 	yield{}：yield方法为生成器，迭代地将数据返回结果写入字典中；
# 	strip()：strip方法用于去掉字符串中多余的空格。
def parse_page(html):
    soup = BeautifulSoup(html, 'lxml')
    items = soup.find_all("div", class_="comment")
    for item in items:
        info = item.find_all(name='span', attrs={"class": "comment-info"})
        for users in info:
            user = users.find_all(name='a', attrs={"class": ""})[0].string
        yield{
            "user":user,
            "time":item.find_all(name='span', attrs={"class":"comment-time"})[0].string.strip(),
            "comment":item.find_all(name='span', attrs={"class":"short"})[0].string
        }


In [11]:
for i in parse_page(html):
    print(i)

{'user': '小小明月几时有', 'time': '2019-09-04', 'comment': '漫威:制造一个大坏蛋，首先要给他超高的智慧，强大的肉体，绝顶的战斗技能，对了，还有无限宝石\n\n DC: 给他一个糟糕的人生吧'}
{'user': 'TWY', 'time': '2019-10-04', 'comment': '华金演绎的小丑固然精彩，但从导演不相信小丑可以从化学池里诞生这件事中，大概能看出他并不是很相信影像，所以只好大搞心理现实主义的mumbo jumbo，然而最终还是只能用大量堆砌的刻板印象制造冲突来推动故事发生。'}
{'user': '小油飞', 'time': '2019-10-04', 'comment': '电影语言是精致和饱满的，但内核是空洞和混乱的，it’s just a fucked-up white dude creepily stalking a black woman, killing people, and blaming everything on vague, void, and incoherent “social” problems so that it is intriguing but won’t actually offend the target audience. 那些经过精心计算的愤怒和挑衅都成为了商业价值的一部分。Yet it says absolutely nothing. '}
{'user': '谋杀游戏机', 'time': '2019-08-31', 'comment': '完全不是超英片，就是一个以普通底层小市民为主角的犯罪片，大概也是这些年漫威DC所有电影里人性剖析最深刻的一部。影片对哥谭市有着出色的氛围营造，阴暗潮湿的道路，满目垃圾的街头，一触即发的暴动气息，这样的环境，再加上悲惨的身世、糟糕的一天，足以把一个老实的可怜人逼向深渊，从理性善良一点点变得麻木和疯癫，每一步转变都真实可信。杰昆·菲尼克斯瘦骨嶙峋的样子让人心疼，毫无疑问，影帝级的表演，“希斯·莱杰后再无小丑”的话可以收回了。'}
{'user': '帕拉', 'time': '2019-10-04', 'comment': '还是很常规的好莱坞剧作方式，并没有惊艳，剧作上的递进很明显，情绪上代入感却低，于我而言算